In [1]:
import os

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib import cm, transforms
from mpl_axes_aligner import align
import numpy as np
import pandas as pd
from tqdm import tqdm
from scipy import special
from scipy.stats import norm
from scipy.stats import multivariate_normal
import h5py
import torch
from torchviz import make_dot

import wf_func as wff

In [2]:
def normcombine(x, m, s, a):
    return a[0] * norm.pdf((x - m[0]) / s[0]) + a[1] * norm.pdf((x - m[1]) / s[1])

def normcombine2d(x, m, s, a, rho):
    return a[0, 0] * multivariate_normal.pdf(x, mean=[m[0, 0], m[1, 0]], cov=matrix(s[0, 0], s[1, 0], rho[0, 0])) + a[0, 1] * multivariate_normal.pdf(x, mean=[m[0, 0], m[1, 1]], cov=matrix(s[0, 0], s[1, 1], rho[0, 1])) + a[1, 0] * multivariate_normal.pdf(x, mean=[m[0, 1], m[1, 0]], cov=matrix(s[0, 1], s[1, 0], rho[1, 0])) + a[1, 1] * multivariate_normal.pdf(x, mean=[m[0, 1], m[1, 1]], cov=matrix(s[0, 1], s[1, 1], rho[1, 1]))

def matrix(sx, sy, rho):
    return np.array([[sx ** 2, rho * sx * sy], [rho * sx * sy, sy ** 2]])

def chargehist(t):
    q1 = 150.8
    sigma = 37.59
    w = 2.433e-5
    alpha = 0.01335
    mu = 2.851e-5
    c = np.exp(-mu)*(w*alpha*np.exp(-alpha*t))
    c = c + mu*np.exp(-mu)*(
        (1-w)/(sigma*np.sqrt(2*np.pi))*np.exp(-(t-q1)**2/(2*sigma**2))+
        w*(alpha/2*np.exp(-alpha*(t-q1-alpha/2*sigma**2))*(1+special.erf(t-q1-alpha*sigma**2)/(np.sqrt(2)*sigma))))
    return c

In [3]:
window = 1029
p = [8., 0.5, 24.]
t = np.arange(window).astype(np.float)
fig = plt.figure()
# fig.tight_layout()
gs = gridspec.GridSpec(1, 1, figure=fig, left=0.15, right=0.85, top=0.95, bottom=0.15, wspace=0.4, hspace=0.5)
ax = fig.add_subplot(gs[0, 0])
t = np.arange(0, 100, 0.1)
ax.plot(t, wff.spe(t, p[0], p[1], p[2]), color='b', label='Single PE response')
ax.set_xlabel(r'$t/\mathrm{ns}$')
ax.grid()
ax.set_xlim(0, 80)
ax.set_ylabel(r'$Voltage/\mathrm{mV}$')
ax.legend()
fig.savefig('Note/figures/spe.pgf')
fig.savefig('Note/figures/spe.pdf')
plt.close()

In [4]:
Tau = 10
Sigma = 2
fig = plt.figure()
t = np.arange(-int(Sigma), int(5 * Tau), 0.1)
gs = gridspec.GridSpec(1, 1, figure=fig, left=0.15, right=0.85, top=0.95, bottom=0.15, wspace=0.4, hspace=0.5)
ax = fig.add_subplot(gs[0, 0])
ax.plot(t, wff.convolve_exp_norm(t, Tau, Sigma), label='Time Profile')
ax.set_xlabel(r'$t/\mathrm{ns}$')
ax.grid()
ax.set_xlim(xmin=-3)
ax.set_ylabel(r'$PDF$')
ax.legend()
fig.savefig('Note/figures/profile.pgf')
fig.savefig('Note/figures/profile.pdf')
plt.close()

In [5]:
fig = plt.figure()
# fig.tight_layout()
gs = gridspec.GridSpec(1, 1, figure=fig, left=0.12, right=0.95, top=0.9, bottom=0.15, wspace=0.4, hspace=0.5)
ax = fig.add_subplot(gs[0, 0])
ax.plot(np.arange(500), chargehist(np.arange(500)))
ax.set_xlabel(r'$Charge/\mathrm{mV}\cdot\mathrm{ns}$')
ax.set_xlim(20, 400)
ax.set_ylim(bottom=0)
ax.grid()
fig.savefig('Note/figures/chargehist.pgf')
fig.savefig('Note/figures/chargehist.pdf')
fig.clf()
plt.close(fig)

In [6]:
xn = np.linspace(-5, 5, 600)
yn = np.linspace(-5, 5, 600)
xv, yv = np.meshgrid(xn, yn)
pos = np.dstack((xv, yv))

m = np.array([[-1., 1.5], [-1., 1.]])
s = np.array([[1., .5], [.6, .8]])
a = np.array([[.4, .6], [.4, .6]])
rho = np.array([[0.5, -0.7], [0.4, 0.4]])

ymax = max(np.max(normcombine(xn, m[0, :], s[0, :], a[0, :])), np.max(normcombine(yn, m[1, :], s[1, :], a[1, :])))
fig = plt.figure(figsize=(5, 5))
ax0 = fig.add_axes((.1, .85, .75, .12))
ax0.axis('off')
ax0.plot(xn, normcombine(xn, m[0, :], s[0, :], a[0, :]), color='b')
ax0.fill_between(xn, normcombine(xn, m[0, :], s[0, :], a[0, :]), facecolor='blue', alpha=0.5, color=None)
ax0.set_xticks([])
ax0.set_yticks([])
ax0.set_ylim(0, ymax)
ax1 = fig.add_axes((.85, .1, .12, .75))
rot = transforms.Affine2D().rotate_deg(-90)
base = plt.gca().transData
ax1.axis('off')
ax1.plot(yn, normcombine(yn, m[1, :], s[1, :], a[1, :]), color='b', transform=rot+base)
ax1.fill_between(yn, normcombine(yn, m[1, :], s[1, :], a[1, :]), facecolor='blue', alpha=0.5, color=None, transform=rot+base)
ax1.set_xticks([])
ax1.set_yticks([])
ax1.set_xlim(0, ymax)
ax2 = fig.add_axes((.1, .1, .75, .75))
ax2.contourf(xv, yv, normcombine2d(pos, m, s, a, rho), cmap=cm.PuBu)
ax2.set_xticks([])
ax2.set_yticks([])
ax2.set_xlabel(r'$\mu$')
ax2.set_ylabel(r'$\nu$')
ax2.spines['top'].set_visible(False)
ax2.spines['right'].set_visible(False)
fig.savefig('Note/figures/wdistdemo.pgf')
fig.savefig('Note/figures/wdistdemo.pdf')
fig.clf()
plt.close(fig)

In [7]:
t = np.array([0, 1, 2, 3])
a = np.array([0.5, 0.5, 0.5, 1])
b1 = np.array([0.5, 0.5, 1, 0.5])
b2 = np.array([1, 0.5, 0.5, 0.5])
fig = plt.figure()
# fig.tight_layout()
gs = gridspec.GridSpec(2, 2, figure=fig, left=0.12, right=0.95, top=0.97, bottom=0.15, wspace=0.4, hspace=0.5)
ax0 = fig.add_subplot(gs[0, 0])
ax1 = fig.add_subplot(gs[0, 1])
ax2 = fig.add_subplot(gs[1, 0])
ax3 = fig.add_subplot(gs[1, 1])
ax0.bar(t, a, color='r')
ax0.set_xlabel(r'$a$')
ax0.set_ylabel(r'$weight\ a$')
ax0.set_xticks(t)
ax1.bar(t, b1, color='b')
ax1.set_xlabel(r'$b_{1}$')
ax1.set_ylabel(r'$weight\ b_{1}$')
ax1.set_xticks(t)
ax2.bar(t, b2, color='b')
ax2.set_xlabel(r'$b_{2}$')
ax2.set_ylabel(r'$weight\ b_{2}$')
ax2.set_xticks(t)
ax3.text(0, 0, r'$L_{2}\ =\ 0.25$')
ax3.set_xlim(0, 5)
ax3.set_ylim(-1, 1)
ax3.spines['top'].set_visible(False)
ax3.spines['right'].set_visible(False)
ax3.spines['bottom'].set_visible(False)
ax3.spines['left'].set_visible(False)
ax3.get_xaxis().set_visible(False)
ax3.get_yaxis().set_visible(False)
fig.savefig('Note/figures/tab.pgf')
fig.savefig('Note/figures/tab.pdf')
fig.clf()
plt.close(fig)

In [8]:
spe_pre = wff.read_model('spe.h5')
with h5py.File('waveform/05.0-20-05.0.h5', 'r', libver='latest', swmr=True) as ipt:
    ent = ipt['Readout/Waveform'][:]
    tru = ipt['SimTriggerInfo/PEList'][:]

In [9]:
i = 1
cid = ent[i]['ChannelID']
eid = ent[i]['TriggerNo']
truth = np.sort(tru[(tru['TriggerNo'] == eid) & (tru['PMTId'] == cid)], kind='stable', order=['TriggerNo', 'PMTId', 'HitPosInWindow'])
wave = ent[i]['Waveform'].astype(np.float) * spe_pre[ent[i]['ChannelID']]['epulse']
df = pd.DataFrame(truth)
df = df.rename(columns={'HitPosInWindow':'HitTime'})
charge = df['Charge'].copy()
hittime = df['HitTime'].copy()
df = df.astype({'Charge': 'float32'})
df = df.astype({'TriggerNo' : 'str', 'PMTId' : 'str', 'HitTime' : 'str', 'Charge': 'str'})
df['HitTime'] = ['{:.02f}'.format(s) for s in hittime]
df['Charge'] = ['{:.02f}'.format(s) for s in charge]

In [10]:
fig = plt.figure()
# fig.tight_layout()
gs = gridspec.GridSpec(1, 1, figure=fig, left=0.15, right=0.85, top=0.95, bottom=0.15, wspace=0.4, hspace=0.5)
ax = fig.add_subplot(gs[0, 0])
ax2 = ax.twinx()
ax2.vlines(truth['HitPosInWindow'].min(), 0, wave.sum(), color='r', label='Record')
ax.plot(wave, label='Waveform')
ax.hlines(2, 0, 1029, color='g', label='Threshold')
ax.set_xlim(200, 600)
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2)
ax.set_xlabel(r'$t/\mathrm{ns}$')
ax.set_ylabel(r'$Voltage/\mathrm{mV}$')
ax2.set_ylabel(r'$Charge/\mathrm{mV}\cdot\mathrm{ns}$')
rise = truth['HitPosInWindow'].min()
vsum = wave.sum()
ax2.annotate(fr'${rise:.{4}}\mathrm{{ns}}, {vsum:.{4}}\mathrm{{mV}}\cdot\mathrm{{ns}}$', xy=(truth['HitPosInWindow'].min(), wave.sum()*2/3), xytext=(truth['HitPosInWindow'].min()+120, wave.sum()*1/2), arrowprops=dict(facecolor='k', shrink=0.1, width=0.1, headwidth=2))
ax.set_ylim(bottom=-5)
ax2.set_ylim(bottom=-5)
align.yaxes(ax, 0, ax2, 0)
fig.savefig('Note/figures/previous.pgf')
fig.savefig('Note/figures/previous.pdf')
fig.clf()
plt.close(fig)

In [11]:
fig = plt.figure()
# fig.tight_layout()
gs = gridspec.GridSpec(1, 1, figure=fig, left=0.15, right=0.85, top=0.95, bottom=0.15, wspace=0.4, hspace=0.5)
ax = fig.add_subplot(gs[0, 0])
ax.plot(wave, label='Waveform')
ax.set_xlabel(r'$t/\mathrm{ns}$')
ax.set_ylabel(r'$Voltage/\mathrm{mV}$')
ax.set_xlim(0, len(wave))
ax.set_ylim(bottom=-5)
ax.legend()
fig.savefig('Note/figures/wave.pgf')
fig.savefig('Note/figures/wave.pdf')
fig.clf()
plt.close(fig)

fig = plt.figure()
# fig.tight_layout()
gs = gridspec.GridSpec(1, 1, figure=fig, left=0.15, right=0.85, top=0.95, bottom=0.15, wspace=0.4, hspace=0.5)
ax = fig.add_subplot(gs[0, 0])
ax.vlines(truth['HitPosInWindow'], 0, truth['Charge'], color='r', label='Charge')
ax.set_xlabel(r'$t/\mathrm{ns}$')
ax.set_ylabel(r'$Charge/\mathrm{mV}\cdot\mathrm{ns}$')
ax.set_xlim(0, len(wave))
ax.legend()
fig.savefig('Note/figures/charge.pgf')
fig.savefig('Note/figures/charge.pdf')
fig.clf()
plt.close(fig)

In [12]:
fig = plt.figure()
# fig.tight_layout()
gs = gridspec.GridSpec(1, 1, figure=fig, left=0.15, right=0.85, top=0.95, bottom=0.15, wspace=0.4, hspace=0.5)
ax = fig.add_subplot(gs[0, 0])
ax2 = ax.twinx()
ax2.vlines(truth['HitPosInWindow'], 0, truth['Charge'], color='r', label='Charge')
ax.plot(wave, label='Waveform')
ax.hlines(2, 0, 1029, color='g', label='Threshold')
ax.set_xlim(200, 600)
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2)
ax.set_xlabel(r'$t/\mathrm{ns}$')
ax.set_ylabel(r'$Voltage/\mathrm{mV}$')
ax2.set_ylabel(r'$Charge/\mathrm{mV}\cdot\mathrm{ns}$')
ax.set_ylim(bottom=-5)
ax2.set_ylim(bottom=-5)
align.yaxes(ax, 0, ax2, 0)
fig.savefig('Note/figures/goal.pgf')
fig.savefig('Note/figures/goal.pdf')
fig.clf()
plt.close(fig)

In [13]:
print(wave.sum())
print(truth['Charge'][truth['Charge'] > 0].sum())
truth['Charge']

1181.0
697.5019906924377


array([149.06832534, 176.25533474, 180.54215478, 191.63617584])

In [14]:
pet, pwe = wff.threshold(wave, spe_pre[cid])
pet, pwe = wff.clip(pet, pwe, 0.1)
pwe = pwe / pwe.sum() * np.abs(wave.sum())
fig = plt.figure()
# fig.tight_layout()
ax = fig.add_subplot(111)
ax2 = ax.twinx()
ax2.vlines(pet, 0, pwe, color='r', label='Charge', linewidth=0.5)
ax.plot(wave, label='Waveform')
ax.hlines(5 * spe_pre[cid]['std'], 0, 1029, color='g', label='Threshold')
ax.set_xlim(200, 600)
ax2.annotate('', xy=(pet.mean(), pwe.max()*1.1), xytext=(pet.mean()+pet.ptp(), pwe.max()*1.1), arrowprops=dict(facecolor='k', shrink=0.01, width=2, headwidth=4))
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2)
ax.set_xlabel(r'$t/\mathrm{ns}$')
ax.set_ylabel(r'$Voltage/\mathrm{mV}$')
ax2.set_ylabel(r'$Charge/\mathrm{mV}\cdot\mathrm{ns}$')
align.yaxes(ax, 0, ax2, 0)
fig.savefig('Note/figures/threshold.pgf')
fig.savefig('Note/figures/threshold.pdf')
fig.clf()
plt.close(fig)
wff.demo(pet, pwe, truth, spe_pre[cid], 1029, wave, cid)

PEnum is 4
truth HitPosInWindow = [321.1045072  327.61275399 330.65613852 334.51873262], Weight = [149.06832534 176.25533474 180.54215478 191.63617584]
truth Resi-norm = 290.1894272410424
HitPosInWindow = [323 324 325 326 327 328 329 330 331 332 333 334 335 336 337 338 339 340], Weight = [35.74918919 40.85621622 48.51675676 54.90054054 65.11459459 70.22162162
 76.60540541 82.98918919 86.81945946 91.92648649 90.64972973 86.81945946
 79.15891892 67.66810811 63.83783784 53.62378378 45.96324324 39.57945946]
wdist = 2.7924853826058387, cdiff = 483.49800930756226
Resi-norm = 70.05292624690911


In [15]:
pet, pwe = wff.findpeak(wave, spe_pre[cid])
fig = plt.figure()
# fig.tight_layout()
ax = fig.add_subplot(111)
ax2 = ax.twinx()
ax2.vlines(pet, 0, pwe, color='r', label='Charge', linewidth=1.5)
ax.plot(wave, label='Waveform')
ax.hlines(5 * spe_pre[cid]['std'], 0, 1029, color='g', label='Threshold')
ax.set_xlim(200, 600)
loc = pet
amp = pwe
for i in range(len(loc)):
    ax.annotate('', xy=(loc[i], amp[i]+5), xytext=(loc[i], amp[i]+35), arrowprops=dict(facecolor='k', shrink=0.01, width=0.5, headwidth=2))
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2)
ax.set_xlabel(r'$t/\mathrm{ns}$')
ax.set_ylabel(r'$Voltage/\mathrm{mV}$')
ax2.set_ylabel(r'$Charge/\mathrm{mV}\cdot\mathrm{ns}$')
align.yaxes(ax, 0, ax2, 0)
fig.savefig('Note/figures/findpeak.pgf')
fig.savefig('Note/figures/findpeak.pdf')
fig.clf()
plt.close(fig)
wff.demo(pet, pwe, truth, spe_pre[cid], 1029, wave, cid)

PEnum is 4
truth HitPosInWindow = [321.1045072  327.61275399 330.65613852 334.51873262], Weight = [149.06832534 176.25533474 180.54215478 191.63617584]
truth Resi-norm = 290.1894272410424
HitPosInWindow = [333], Weight = [4.33122217]
wdist = 4.9275586265633455, cdiff = -693.1707685236426
Resi-norm = 237.28982190317618


In [16]:
pet, pwe = wff.waveformfft(wave, spe_pre[cid])
pet, pwe = wff.clip(pet, pwe, 0.1)
pwe = pwe / pwe.sum() * np.abs(wave.sum())
fig = plt.figure()
# fig.tight_layout()
ax = fig.add_subplot(111)
ax2 = ax.twinx()
ax2.vlines(pet, 0, pwe, color='r', label='Charge', linewidth=0.5)
ax.plot(wave, label='Waveform')
ax.hlines(5 * spe_pre[cid]['std'], 0, 1029, color='g', label='Threshold')
ax.set_xlim(200, 600)
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2)
ax.set_xlabel(r'$t/\mathrm{ns}$')
ax.set_ylabel(r'$Voltage/\mathrm{mV}$')
ax2.set_ylabel(r'$Charge/\mathrm{mV}\cdot\mathrm{ns}$')
align.yaxes(ax, 0, ax2, 0)
fig.savefig('Note/figures/fftrans.pgf')
fig.savefig('Note/figures/fftrans.pdf')
fig.clf()
plt.close(fig)
wff.demo(pet, pwe, truth, spe_pre[cid], 1029, wave, cid)

PEnum is 4
truth HitPosInWindow = [321.1045072  327.61275399 330.65613852 334.51873262], Weight = [149.06832534 176.25533474 180.54215478 191.63617584]
truth Resi-norm = 290.1894272410424
HitPosInWindow = [319 320 321 322 326 327 328 329 330 331 332 333 334 335 336], Weight = [ 49.90566324  76.92552701  75.5194317   48.97616005  47.22796816
  82.96148769 102.60214098 100.99538404  87.75599052  78.47303328
  82.23494604  94.76163222 101.95382984  90.87173294  59.83507228]
wdist = 0.9352272256787972, cdiff = 483.49800930756226
Resi-norm = 36.11550312944896


In [17]:
pet, pwe = wff.lucyddm(wave, spe_pre[cid]['spe'])
pet, pwe = wff.clip(pet, pwe, 0.1)
pwe = pwe / pwe.sum() * np.abs(wave.sum())
fig = plt.figure()
# fig.tight_layout()
ax = fig.add_subplot(111)
ax2 = ax.twinx()
ax2.vlines(pet, 0, pwe, color='r', label='Charge', linewidth=0.5)
ax.plot(wave, label='Waveform')
ax.hlines(5 * spe_pre[cid]['std'], 0, 1029, color='g', label='Threshold')
ax.set_xlim(200, 600)
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2)
ax.set_xlabel(r'$t/\mathrm{ns}$')
ax.set_ylabel(r'$Voltage/\mathrm{mV}$')
ax2.set_ylabel(r'$Charge/\mathrm{mV}\cdot\mathrm{ns}$')
align.yaxes(ax, 0, ax2, 0)
fig.savefig('Note/figures/lucyddm.pgf')
fig.savefig('Note/figures/lucyddm.pdf')
fig.clf()
plt.close(fig)
wff.demo(pet, pwe, truth, spe_pre[cid], 1029, wave, cid)

PEnum is 4
truth HitPosInWindow = [321.1045072  327.61275399 330.65613852 334.51873262], Weight = [149.06832534 176.25533474 180.54215478 191.63617584]
truth Resi-norm = 290.1894272410424
HitPosInWindow = [320 321 322 326 327 328 329 330 331 332 333 334 335 336], Weight = [ 50.6856103  117.38931547  66.68201817  39.79450763  61.65765477
  86.9010258  104.11272231 110.91665042 117.54480744 117.39255013
 112.85084456  95.16619182  62.97684966  36.92925154]
wdist = 0.9339246933134131, cdiff = 483.49800930756203
Resi-norm = 36.60672397387282


In [18]:
pet, pwe0 = wff.xiaopeip(wave, spe_pre[cid])
pet, pwe0 = wff.clip(pet, pwe0, 0.1)
pwe = pwe0 * spe_pre[cid]['spe'].sum()
wff.demo(pet, pwe, truth, spe_pre[cid], 1029, wave, cid)

PEnum is 4
truth HitPosInWindow = [321.1045072  327.61275399 330.65613852 334.51873262], Weight = [149.06832534 176.25533474 180.54215478 191.63617584]
truth Resi-norm = 290.1894272410424
HitPosInWindow = [321 327 328 330 331 332 334 335], Weight = [240.54001475 147.27504881 126.45573278 107.07545018 163.09905943
  41.91916538 189.41485894 134.06784897]
wdist = 0.5190266576262166, cdiff = 452.3451885513549
Resi-norm = 36.16212627472232


In [19]:
s = [r'$\mathrm{FindPeak}$', r'$\mathrm{Threshold}$', r'$\mathrm{FFT}$', r'$\mathrm{LucyDDM}$', r'$\mathrm{CNN}$', r'$\mathrm{Fitting}$']
x = np.arange(0, 6)
y = np.array([10.71, 3.67, 1.45, 0.93, 0.248, 0.504])
dy = np.array([[1.32, 1.80, 0.86, 0.49, 0.065, 0.16], [24.06, 6.11, 2.12, 1.41, 0.425, 0.87]])
dy[0, :] = y - dy[0, :]
dy[1, :] = dy[1, :] - y
fig = plt.figure()
# fig.tight_layout()
ax = fig.add_subplot(111)
ax.bar(x, y, color='b')
ax.set_ylim(0, 26)
ax.set_ylabel(r'$\mathrm{Wasserstein}\ \mathrm{Distance}/\mathrm{ns}$')
ax.set_xticks(x)
ax.set_xticklabels(s)
ax.errorbar(x, y, yerr=dy, fmt='o', ecolor='r', color='b', elinewidth=2)
fig.savefig('Note/figures/summarycharge.pgf')
fig.savefig('Note/figures/summarycharge.pdf')
fig.clf()
plt.close(fig)

In [20]:
with h5py.File('result/lucyddm/dist/05.0-20-05.0.h5', 'r', libver='latest', swmr=True) as distfile:
    dt = distfile['Record'][:]

In [21]:
fig = plt.figure()
# fig.tight_layout()
gs = gridspec.GridSpec(1, 1, figure=fig, left=0.15, right=0.85, top=0.95, bottom=0.15, wspace=0.4, hspace=0.5)
ax = fig.add_subplot(gs[0, 0])
ax.hist(dt['TotalPEpos'], bins=np.max(dt['TotalPEpos']), histtype='step')
ax.set_yscale('log')
ax.grid()
ax.set_xlabel(r'$N_{pos}$')
ax.set_ylabel(r'$Count$')
fig.savefig('Note/figures/pepos.pgf')
fig.savefig('Note/figures/pepos.pdf')
plt.close(fig)

In [22]:
methods = ['lucyddm']
# methods = ['xiaopeip', 'takara']

for m in methods:
    with h5py.File('result/' + m + '/dist/05.0-20-05.0.h5', 'r', libver='latest', swmr=True) as distfile:
        dt = distfile['Record'][:]
    N = np.percentile(dt['wdist'], 95)
    M = 500

    penum = np.unique(dt['TotalPEpos'])
    l = min(50, penum.max())
    wdist_stats = np.full((l, 6), np.nan)
    edist_stats = np.full((l, 6), np.nan)
    for i in range(l):
        vali = dt['TotalPEpos'] == i+1
        if np.sum(vali) == 0:
            continue
        dtwpi = dt['wdist'][vali]
        dtepi = dt['RSS'][vali]
        wdist_stats[i, 0] = np.median(dtwpi)
        wdist_stats[i, 1] = np.median(np.abs(dtwpi - np.median(dtwpi)))
        wdist_stats[i, 2] = np.mean(dtwpi)
        wdist_stats[i, 3] = np.std(dtwpi)
        wdist_stats[i, 4] = np.percentile(dtwpi, 5)
        wdist_stats[i, 5] = np.percentile(dtwpi, 95)
        edist_stats[i, 0] = np.median(dtepi)
        edist_stats[i, 1] = np.median(np.abs(dtepi - np.median(dtepi)))
        edist_stats[i, 2] = np.mean(dtepi)
        edist_stats[i, 3] = np.std(dtepi)
        edist_stats[i, 4] = np.percentile(dtepi, 5)
        edist_stats[i, 5] = np.percentile(dtepi, 95)

    n = 4 # corresponding with Algorithm.tex
    dtn = dt[dt['TotalPEpos'] == n]
    a = (dtn['wdist'] < N).sum()
    b = (dtn['RSS'] < M).sum()
    L = len(dtn['wdist'])
    fig = plt.figure()
    ax = fig.add_axes((.1, .45, .85, .45))
    data = dtn['wdist'][dtn['wdist']<N]
    ax.hist(data, bins=50, density=1)
    ax.set_ylabel(r'$Normalized\ Count$')
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xlim(data.min() - 0.05, data.max() + 0.05)
    # ax.set_title('count {}(Wd<{:.2f}ns)/{}={:.2f}'.format(a, N, L, a/L))
    axb = fig.add_axes((.1, .15, .85, .3))
    axb.boxplot(dtn['wdist'][dtn['wdist']<N], vert=False, sym='', patch_artist=True)
    axb.set_xlabel(r'$W-dist/\mathrm{ns}$')
    axb.set_yticks([])
    axb.set_xlim(ax.get_xlim())
    fig.savefig('Note/figures/' + m + 'chargehist.pgf')
    fig.savefig('Note/figures/' + m + 'chargehist.pdf')
    plt.close(fig)

    fig = plt.figure()
    gs = gridspec.GridSpec(1, 1, figure=fig, left=0.15, right=0.85, top=0.95, bottom=0.15, wspace=0.4, hspace=0.5)
    ax = fig.add_subplot(gs[0, 0])
    ey = np.vstack([wdist_stats[:, 0]-wdist_stats[:, 4], wdist_stats[:, 5]-wdist_stats[:, 0]])
    ax.errorbar(np.arange(1, l+1), wdist_stats[:, 0], yerr=ey, label=r'$W-dist^{95\%}_{5\%}$')
    ax.set_xlim(-1, l)
    ax.set_xlabel(r'$N_{pos}$')
    ax.set_ylabel(r'$W-dist/\mathrm{ns}$')
    # ax.set_title('W-dist vs TotalPEpos stats')
    ax.legend()
    fig.savefig('Note/figures/' + m + 'chargestats.pgf')
    fig.savefig('Note/figures/' + m + 'chargestats.pdf')
    plt.close(fig)